For the Reddit posts and comments, I forgot to remove documents that aren't about our three target types (pill, iud, implant). So while Roz labeled 300 comments/posts, and LeAnn and I labeled 500, we actually only labeled 249 posts and 198 comments.

For now, I'll just sample one post and 52 comments so that we have at least 250 for each set.

In [1]:
from collections import defaultdict
from datetime import datetime
import dill
from itertools import permutations, combinations
import json
import jsonlines
from operator import itemgetter
import os
import pickle
import random
import re
import time

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
sns.set(style='ticks', font_scale=1.2)
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import little_mallet_wrapper as lmw
import octis

In [2]:
data_directory_path   = '/Volumes/Passport-1/data/birth-control'
output_directory_path = '/Volumes/Passport-1/output/birth-control'

In [3]:
reddit_posts_df = pd.read_csv(data_directory_path + '/final-data/reddit_posts.csv')
reddit_comments_df = pd.read_csv(data_directory_path + '/final-data/reddit_comments.csv')
len(reddit_posts_df.index), len(reddit_comments_df.index)

(72731, 238568)

In [4]:
maria_df = pd.read_csv(data_directory_path + '/labeling/labeled_by_maria.all.csv')
len(maria_df)

2495

In [5]:
used_ids = list(set(maria_df['ID'].tolist()))
len(used_ids)

1977

In [6]:
reddit_posts_df = reddit_posts_df[~reddit_posts_df['id'].isin(used_ids)]
len(reddit_posts_df)

72319

In [7]:
reddit_comments_df = reddit_comments_df[~reddit_comments_df['id'].isin(used_ids)]
len(reddit_comments_df)

238240

In [8]:
sampled_df = pd.concat([reddit_posts_df.sample(1), reddit_comments_df.sample(52)])
len(sampled_df)

53

In [9]:
sampled_df.sample(5)

,Unnamed: 0,id,created_utc,text,title,year,month,url,link_flair_text,source,tokens_text,text_type,Unnamed: 0.1,parent_id
75220,75220,dfwq8yy,1.491488e+09,"I've not had an abortion, but I would suggest ...",NaN,2017,4,NaN,NaN,reddit-comments,abortion would suggest basics would period ill...,pill,3726.0,t3_63qguf
214312,214312,fpdekpl,1.588525e+09,How long have you had it? I'd give it time. My...,NaN,2020,5,NaN,NaN,reddit-comments,long d give time first periods pretty painful ...,iud,10644.0,t3_gcs1d4
105138,105138,dvaqlha,1.520386e+09,I have the implant and I really love it. I had...,NaN,2018,3,NaN,NaN,reddit-comments,implant really love side effects pill depresse...,implant,4067.0,t3_82j6a2
164586,164586,es3z7h0,1.561574e+09,"I'm sorry you had such a scary experience, but...",NaN,2019,6,NaN,NaN,reddit-comments,m sorry scary experience take deep breath rock...,iud,1111.0,t3_c5ebhw
85554,85554,dncx6cb,1.506095e+09,As long as your break is only 7 days or less a...,NaN,2017,9,NaN,NaN,reddit-comments,long break NUM days less start new pack time r...,pill,1282.0,t1_dncvzs0


In [10]:
def get_data_dicts(df):
    data_dicts = []
    for i, r in df.iterrows():
        if 'title' in r and not pd.isnull(r['title']):
            data_dicts.append({'text': '[TITLE: ' + r['title'] + '] \n\n' + r['text'],
                               'meta': {'ID': r['id'],
                                        'Source': r['source']}})
        else:
            data_dicts.append({'text': r['text'],
                               'meta': {'ID': r['id'],
                                        'Source': r['source']}})
    return data_dicts


dicts_to_label = get_data_dicts(sampled_df)
len(dicts_to_label)

53

In [11]:
random.sample(dicts_to_label, 1)[0]

{'text': 'Lol, one pill is enough.',
 'meta': {'ID': 'eljs5ck', 'Source': 'reddit-comments'}}

In [13]:
with jsonlines.open(data_directory_path + '/labeling/sampled-reddit-extra.prodigy.jsonl', 'w') as writer:
    writer.write_all(dicts_to_label)